In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GraphConv  # Import GraphConv
from torch.optim import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('Dataset.csv')

In [5]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,MACD,RSI,MA,EMA,SMA,STD,Upper,Lower,numeric_classes
0,6/13/2018,48.105000,48.220001,47.610001,47.674999,45.561813,86553600,0.343733,57.884069,47.404500,47.518221,47.404500,0.621977,48.648453,46.160546,4
1,6/14/2018,47.887501,47.892502,47.555000,47.700001,45.585709,86440400,0.315295,58.995271,47.437250,47.535534,47.437250,0.619290,48.675829,46.198670,0
2,6/15/2018,47.507500,47.540001,47.064999,47.209999,45.117435,246876800,0.250333,50.799494,47.460375,47.504530,47.460375,0.600529,48.661433,46.259316,0
3,6/18/2018,46.970001,47.305000,46.799999,47.185001,45.093540,73939600,0.194590,52.678608,47.490750,47.474099,47.490750,0.568006,48.626761,46.354739,0
4,6/19/2018,46.285000,46.582500,45.862499,46.422501,44.364834,134314000,0.087873,45.062755,47.466500,47.373947,47.466500,0.603463,48.673427,46.259573,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233,05-08-2023,172.479996,173.850006,172.110001,173.500000,173.260345,55962800,2.895528,65.079344,166.603000,166.920041,166.603000,3.441849,173.486697,159.719302,0
1234,05-09-2023,173.050003,173.539993,171.600006,171.770004,171.532745,45326900,2.933239,58.668332,167.151500,167.381942,167.151500,3.341023,173.833546,160.469454,0
1235,05-10-2023,173.020004,174.029999,171.899994,173.559998,173.320267,53724500,3.072149,63.993525,167.824500,167.970328,167.824500,3.198461,174.221422,161.427577,0
1236,05-11-2023,173.850006,174.589996,172.169998,173.750000,173.510010,49514700,3.161129,68.774172,168.234000,168.520773,168.234000,3.410531,175.055062,161.412937,3


In [6]:
sequence_length = 2


In [7]:
def prepare_data(data, sequence_length):
    data = data[['Open', 'High', 'Low', 'Close']].values
    scaler = MinMaxScaler()
    data = scaler.fit_transform(data)
    
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length])
    
    X = np.array(X)
    y = np.array(y)
    
    return X, y


In [8]:
X, y = prepare_data(df, sequence_length)

In [9]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
X_train

array([[[0.82555324, 0.84342365, 0.82829697, 0.84692327],
        [0.83619188, 0.85024912, 0.85036909, 0.85395578]],

       [[0.26572782, 0.26479419, 0.27064479, 0.26662118],
        [0.26115866, 0.27083476, 0.27008775, 0.27481438]],

       [[0.73171473, 0.75162106, 0.7504526 , 0.75652477],
        [0.74514946, 0.75844653, 0.76242866, 0.76123579]],

       ...,

       [[0.72421317, 0.729916  , 0.72761456, 0.71958696],
        [0.72537251, 0.73093988, 0.73527371, 0.73126225]],

       [[0.77761108, 0.77605627, 0.78227273, 0.76724419],
        [0.77038226, 0.77127835, 0.78108896, 0.76690278]],

       [[0.76526752, 0.76923079, 0.7736388 , 0.76990698],
        [0.7499233 , 0.76151797, 0.76695445, 0.76649314]]])

In [11]:
X_test

array([[[0.09486139, 0.09611972, 0.09948126, 0.09949645],
        [0.09724828, 0.11333697, 0.1017964 , 0.11398822]],

       [[0.5894568 , 0.58924306, 0.59218776, 0.59306994],
        [0.59061612, 0.59327006, 0.59908094, 0.58876847]],

       [[0.82739457, 0.83932832, 0.84277954, 0.84480671],
        [0.84089746, 0.85803015, 0.85162235, 0.83490659]],

       ...,

       [[0.2072493 , 0.20537845, 0.20609942, 0.20653752],
        [0.20409521, 0.2018463 , 0.20745717, 0.20452336]],

       [[0.55017558, 0.56159988, 0.56364016, 0.55524455],
        [0.5554949 , 0.55702682, 0.55068931, 0.54759752]],

       [[0.76519926, 0.82635996, 0.78206386, 0.82063674],
        [0.79902479, 0.80410896, 0.81061135, 0.80425023]]])

In [12]:
X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train).view(-1, 1, 4)  # Reshape y_train
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test).view(-1, 1, 4)  # Reshape y_test


In [13]:
X_train

tensor([[[0.8256, 0.8434, 0.8283, 0.8469],
         [0.8362, 0.8502, 0.8504, 0.8540]],

        [[0.2657, 0.2648, 0.2706, 0.2666],
         [0.2612, 0.2708, 0.2701, 0.2748]],

        [[0.7317, 0.7516, 0.7505, 0.7565],
         [0.7451, 0.7584, 0.7624, 0.7612]],

        ...,

        [[0.7242, 0.7299, 0.7276, 0.7196],
         [0.7254, 0.7309, 0.7353, 0.7313]],

        [[0.7776, 0.7761, 0.7823, 0.7672],
         [0.7704, 0.7713, 0.7811, 0.7669]],

        [[0.7653, 0.7692, 0.7736, 0.7699],
         [0.7499, 0.7615, 0.7670, 0.7665]]])

In [14]:
X_test

tensor([[[0.0949, 0.0961, 0.0995, 0.0995],
         [0.0972, 0.1133, 0.1018, 0.1140]],

        [[0.5895, 0.5892, 0.5922, 0.5931],
         [0.5906, 0.5933, 0.5991, 0.5888]],

        [[0.8274, 0.8393, 0.8428, 0.8448],
         [0.8409, 0.8580, 0.8516, 0.8349]],

        ...,

        [[0.2072, 0.2054, 0.2061, 0.2065],
         [0.2041, 0.2018, 0.2075, 0.2045]],

        [[0.5502, 0.5616, 0.5636, 0.5552],
         [0.5555, 0.5570, 0.5507, 0.5476]],

        [[0.7652, 0.8264, 0.7821, 0.8206],
         [0.7990, 0.8041, 0.8106, 0.8043]]])

In [15]:
class GraphConvModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphConvModel, self).__init__()
        self.conv1 = GraphConv(in_channels, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, out_channels)
    
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = self.conv2(x, edge_index)
        return x

In [16]:
num_nodes = sequence_length
edge_index = torch.tensor([[i, j] for i in range(num_nodes) for j in range(num_nodes) if i != j], dtype=torch.long).t().contiguous()


In [17]:
edge_index

tensor([[0, 1],
        [1, 0]])

In [18]:
num_nodes

2

In [19]:
model = GraphConvModel(in_channels=4, hidden_channels=8, out_channels=4)
optimizer = Adam(model.parameters(), lr=0.001)

In [20]:
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    out = model(X_train, edge_index)
    # Reshape the target labels to match the model's output shape
    y_train_reshaped = y_train.view(-1, 1, 4)
    loss = F.mse_loss(out, y_train_reshaped)
    loss.backward()
    optimizer.step()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_2344\502600674.py:7: UserWarning: Using a target size (torch.Size([988, 1, 4])) that is different to the input size (torch.Size([988, 2, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(out, y_train_reshaped)


In [21]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test, edge_index)

In [22]:
y_pred.shape

torch.Size([248, 2, 4])

In [23]:
y_test.shape

torch.Size([248, 1, 4])

In [24]:
rmse = torch.sqrt(F.mse_loss(y_pred, y_test))
print(f'RMSE on the test set: {rmse.item():.4f}')

RMSE on the test set: 0.2095


C:\Users\ASUS\AppData\Local\Temp\ipykernel_2344\1804030440.py:1: UserWarning: Using a target size (torch.Size([248, 1, 4])) that is different to the input size (torch.Size([248, 2, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  rmse = torch.sqrt(F.mse_loss(y_pred, y_test))
